In [1]:
import numpy as np
import pandas as pd
import boto3
import json
import time
from datetime import datetime
from io import StringIO
import botocore
import logging
import io
from stream_iterator import *

sagemaker = boto3.client("runtime.sagemaker")

In [2]:
#CHANGE THIS (create schema of your own tables, columns and types, for text2sql output)
# GSA Fleet
# Schema
telematics_schema = f"CREATE TABLE telematics_story_final_v1 (datetime DATETIME, deviceid STRING, crankingvoltage FLOAT, coolanttemperature FLOAT, oilliferemaining FLOAT, enginetransmissionoiltemperature FLOAT, oilpressure FLOAT)"
predictions_schema = f"CREATE TABLE predictions_story_final_v1 (datetime DATETIME, deviceid STRING, risk_level STRING);"
maintenance_schema = f"CREATE TABLE maintenance_story_final_v1 (datetime DATETIME, deviceid STRING, maintenance_required STRING, make STRING, model STRING)"
tax_exempt_schema = f"CREATE TABLE tax_exempt_final_v1 (EIN STRING, NAME STRING, ICO STRING, STREET STRING, CITY STRING, STATE STRING, ZIP_CODE STRING, GROUP STRING, SUBSECTION STRING, AFFILIATION STRING, CLASSIFICATION STRING, RULING STRING, DEDUCTIBILITY STRING, FOUNDATION STRING, ACTIVITY STRING, ORGANIZATION STRING, STATUS STRING, TAX_PERIOD STRING, ASSET_CD STRING, INCOME_CD STRING, FILING_REQ_CD STRING, PF_FILING_REQ_CD STRING, ACCT_PD STRING, ASSET_AMT FLOAT, INCOME_AMT FLOAT, REVENUE_AMT FLOAT, NTEE_CD STRING, SORT_NAME STRING)"

glue_db = 'deepfleet'

In [3]:
schema = f"\n {telematics_schema} \n {predictions_schema} \n {maintenance_schema} \n"

In [4]:
#CHANGE THIS based on what table your querying
question = "What makes and models have been serviced the most?"

In [5]:
#CHANGE THIS (if required) with information about your data
system_prompt = "-- the maintenance table details the date, vehicleid, and maintenance item needed. the maintenance_required field is not yes or no, it is maintenance items. \
                 -- do not select all columns. only select the metric that was asked for, the datetime, and the deviceid. \
                 -- when applicable, order records from least to most recent. \
                 -- the possible values for the risk_level column in the predictions table are High, Medium, Low. \
                 -- temperatures are all in fahrenheit, not celcius. \
                 -- when asked about problems, the user is referring to cranking voltage"

In [6]:
prompt = f"<s>[INST] <<SYS>> Your job is to translate a provided user question to a SQL query based on the following schemas: {schema}. You must only return one SQL query and no extra text. You must also remember to provide an alias to columns when necessary. \
        {system_prompt} <<SYS>> \n \n {question}. [/INST]"

In [7]:
payload = {
            "inputs": prompt,
            "parameters": {
            "do_sample": True,
            "top_p": 0.5,
            "temperature": 0.1,
            "top_k": 40,
            "max_new_tokens": 200,
            "repetition_penalty": 1.03,
            "stop": ["<|endoftext|>"]
        },
        "stream": True
        }
b = json.dumps(payload).encode()

In [8]:
query=''
stop_token = '</s>'
resp = sagemaker.invoke_endpoint_with_response_stream(EndpointName='text2sql-alpha-stream-v2', Body=json.dumps(payload), ContentType='application/json')
event_stream = resp['Body']
start_json = b'{'
for line in LineIterator(event_stream):
    if line != b'' and start_json in line:
        data = json.loads(line[line.find(start_json):].decode('utf-8'))
        if data['token']['text'] != stop_token:
            query += data['token']['text']
            print(data['token']['text'], end='')


SELECT make, model, COUNT(maintenance_required) AS service_count FROM maintenance_story_final_v1 GROUP BY make, model ORDER BY service_count DESC;

In [9]:
pip install redshift_connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 42.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 13.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install awswrangler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 22.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import redshift_connector
import awswrangler as wr
import pandas as pd
import matplotlib.pyplot as plt

report_bucket = "query-output-df-v2"

Matplotlib is building the font cache; this may take a moment.


In [13]:
query

'\nSELECT make, model, COUNT(maintenance_required) AS service_count FROM maintenance_story_final_v1 GROUP BY make, model ORDER BY service_count DESC;'

In [16]:
# Set up the AWS session
session = boto3.Session()

# Get temporary credentials
credentials = session.get_credentials()

conn_params = {
    'host': 'default-workgroup.496477444687.us-east-1.redshift-serverless.amazonaws.com',
    'port': 5439,
    'database': 'dev',
    'user': 'serverlessuser',
    'db_user': 'serverlessuser',
    'iam': True,
    'region':'us-east-1',    
    'ssl': True,
    #'cluster_identifier': 'default-workgroup',  # This might need adjustment
    'region': 'us-east-1',
    'timeout': 120  # Increase timeout to 120 seconds
}

try:
    print("runs")
    conn = redshift_connector.connect(**conn_params)
    print("Connection successful!")
    
    # Your query execution code here
    df = wr.redshift.read_sql_query(sql=query, con=conn)
    
    # Print the first few rows of the dataframe
    print(df.head())

    conn.close()
except Exception as e:
    print(f"Connection failed: {str(e)}")

runs
Connection failed: ('connection time out', TimeoutError('timed out'))


In [17]:
import awswrangler as wr

# Connection parameters
conn_params = {
    #'host': 'default-workgroup.496477444687.us-east-1.redshift-serverless.amazonaws.com:5439/dev',  # Update with your workgroup endpoint
    #'port': 5439,
    #'database': 'dev',
    'secret_id': 'redshiftsecert',
    #'iam': True,
    #'ssl': True,
    #'timeout': 120
}

try:
    print("Connecting to Redshift Serverless with IAM...")
    # Connect using awswrangler with IAM
    conn = wr.redshift.connect(**conn_params)
    with conn.cursor() as cursor:
        cursor.execute("SELECT 1")
        print(cursor.fetchall())
    print("Connection successful!")

    # Example query
    query = "SELECT * FROM your_table LIMIT 10;"
    
    # Use awswrangler to execute the query
    df = wr.redshift.read_sql_query(sql=query, con=conn)
    
    # Print the first few rows of the dataframe
    print(df.head())

    conn.close()
except Exception as e:
    print(f"Connection failed: {str(e)}")


Connecting to Redshift Serverless with IAM...


KeyboardInterrupt: 

In [18]:
!pip install --upgrade awswrangler

In [20]:
import awswrangler as wr

# Connection parameters
conn_params = {
    'host': 'default-workgroup.496477444687.us-east-1.redshift-serverless.amazonaws.com',
    'port': 5439,
    'database': 'dev',
    'secret_id': 'arn:aws:secretsmanager:us-east-1:496477444687:secret:redshiftsecert-NWrU7o',
    'iam': True,
    'ssl': True,
    'timeout': 120
}

try:
    print("Connecting to Redshift Serverless with IAM...")
    # Connect using awswrangler with secret_id
    conn = wr.redshift.connect(
        host='default-workgroup.496477444687.us-east-1.redshift-serverless.amazonaws.com',
        port=conn_params['port'],
        database=conn_params['database'],
        secret_id=conn_params['secret_id'],
        iam=conn_params['iam'],
        ssl=conn_params['ssl'],
        timeout=conn_params['timeout']
    )
    print("Connection successful!")

    # Example query
    query = "SELECT * FROM your_table LIMIT 10;"
    
    # Use awswrangler to execute the query
    df = wr.redshift.read_sql_query(sql=query, con=conn)
    
    # Print the first few rows of the dataframe
    print(df.head())

    conn.close()
except Exception as e:
    print(f"Connection failed: {str(e)}")


Connecting to Redshift Serverless with IAM...
Connection failed: redshift_connector.connect() got multiple values for keyword argument 'host'


In [17]:
#CHANGE THIS (connect to redshift database and query from that)
df = wr.redshift.read_sql_query(sql=query, con=conn)

conn.close()
except Exception as e:
    print(f"Connection failed: {str(e)}")

IndentationError: unexpected indent (1623043216.py, line 4)

In [ ]:
df

In [21]:
import socket

host = 'default-workgroup.496477444687.us-east-1.redshift-serverless.amazonaws.com'
port = 5439

try:
    socket.create_connection((host, port), timeout=10)
    print("Network connection successful!")
except Exception as e:
    print(f"Network connection failed: {str(e)}")

Network connection failed: timed out


In [ ]:
import requests
print(requests.get('https://checkip.amazonaws.com').text.strip())